## storagy check

In [ ]:
!df -h

In [ ]:
!du -h --max-depth=1 | sort -hr

## Import library

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import time
import random
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# plt.style.use('seaborn')

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Fix seed

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [3]:
seed_everything(0)

## Use cuda

In [4]:
device = torch.device('cuda:0')
torch.cuda.is_available()

True

## Hyper-parameter

In [5]:
max_len = 128
batch_size = 32
warmup_ratio = 0.01
num_epochs = 30
max_grad_norm = 1
learning_rate =  5e-6
stop_count = 5
num_folds = 10
PATH = './model/model_state_dict_init'

## Dataset

In [6]:
class RE_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
def preprocessing_dataset(dataset, label_type):
    label = []
    for i in dataset[8]:
        if i == 'blind':
            label.append(100)
        else:
            label.append(label_type[i])
    out_dataset = pd.DataFrame({'0': dataset[0], 'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
    return out_dataset


In [8]:
def load_data(dataset_dir):
  # load label_type, classes
  with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
  # load dataset
  dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
  # preprecessing dataset
  dataset = preprocessing_dataset(dataset, label_type)
  
  return dataset


In [9]:
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
        temp = ''
        temp = e01 + '</s>' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation='only_second',
        max_length=max_len,
        add_special_tokens=True,
    )
    return tokenized_sentences

In [10]:
df_train = load_data('/opt/ml/input/data/train/train.tsv')

In [11]:
# 10개 미만 label 삭제
df_train = df_train[~df_train['label'].isin((40, 37, 29, 41, 19, 18, 26, 28, 39))]
len(df_train)

8948

## Train

In [13]:
# accuracy 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [14]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=42, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [15]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaConfig

MODEL_NAME = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

In [16]:
model_config = XLMRobertaConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = 42
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config).to(device)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [18]:
# 모델 저장 (fold마다 모델을 불러오기 위해)
torch.save(model.state_dict(), PATH)

In [19]:
t_dataset = tokenized_dataset(df_train, tokenizer)
t_label = df_train['label'].values
dataset = RE_Dataset(t_dataset, t_label)

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# kfold = KFold(n_splits=num_folds, random_state=0, shuffle=True)
kfold = StratifiedKFold(n_splits=num_folds, random_state=0, shuffle=True)
criterion = nn.CrossEntropyLoss()
results = {}

for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset, dataset.labels)):
    print(f'FOLD {fold}')
    print('='*10)
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=train_subsampler,
    )
    
    val_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=val_subsampler,
    )
    
    # 모델을 불러온다. (huggingface로 계속 불러오면 메모리 초과 발생!)
    model.load_state_dict(torch.load(PATH))
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    
    t_total = len(train_loader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)

    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
    
    best_acc = 0.0
    early_stopping_count = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)

        train_acc = 0.0
        val_acc = 0.0

        since = time.time()

        #################### Train ####################
        train_loss = 0.0

        model.train()
        for batch_id, batch in enumerate(train_loader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            train_acc += calc_accuracy(outputs[1], labels)
            train_loss = loss.data.cpu().numpy()

        print(f"train Loss: {train_loss:.4f} Acc: {train_acc/(batch_id+1):.4f}")

        #################### Validation ####################
        val_loss =0.0

        model.eval()
        for batch_id, batch in enumerate(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_acc += calc_accuracy(outputs[1], labels)
            val_loss = outputs[0].data.cpu().numpy()

        print(f"val Loss: {val_loss:.4f} Acc: {val_acc/(batch_id+1):.4f}")

        #################### model save ####################
        if (val_acc/(batch_id+1)) > best_acc:
            early_stopping_count = 0
            print(f"epochs_val acc: {val_acc/(batch_id+1):.4f}")
            print(f"epochs_before_best acc: {best_acc:.4f}")
            best_acc = (val_acc/(batch_id+1))
            print(f"epochs_after_best acc: {best_acc:.4f}")
            torch.save(model.state_dict(), f"/opt/ml/model/model_state_dict{fold}.pt")
        
        else:
            early_stopping_count += 1
        
        #################### running time check ####################
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60}m {time_elapsed % 60}s')
        
        #################### Early Stopping ####################
        if early_stopping_count >= stop_count:
            print('...Early Stopping...')
            print()
            break
        
        print()
    
    results[fold] = best_acc
    print(f'Best val Acc: {best_acc}')
    print()


FOLD 0
Epoch 0/29
----------
train Loss: 2.3571 Acc: 0.4272
val Loss: 1.7743 Acc: 0.4951
epochs_val acc: 0.4951
epochs_before_best acc: 0.0000
epochs_after_best acc: 0.4951
Training complete in 3.0m 48.7668092250824s

Epoch 1/29
----------
train Loss: 1.5470 Acc: 0.5223
val Loss: 0.9225 Acc: 0.6191
epochs_val acc: 0.6191
epochs_before_best acc: 0.4951
epochs_after_best acc: 0.6191
Training complete in 3.0m 48.88650941848755s

Epoch 2/29
----------
train Loss: 1.0525 Acc: 0.6792
val Loss: 0.6754 Acc: 0.7108
epochs_val acc: 0.7108
epochs_before_best acc: 0.6191
epochs_after_best acc: 0.7108
Training complete in 3.0m 48.48181748390198s

Epoch 3/29
----------
train Loss: 0.8265 Acc: 0.7506
val Loss: 0.6722 Acc: 0.7307
epochs_val acc: 0.7307
epochs_before_best acc: 0.7108
epochs_after_best acc: 0.7307
Training complete in 3.0m 48.6054413318634s

Epoch 4/29
----------
train Loss: 0.9903 Acc: 0.7913
val Loss: 1.3741 Acc: 0.7541
epochs_val acc: 0.7541
epochs_before_best acc: 0.7307
epochs_afte

In [21]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value

print(f'Average: {sum/len(results.items())} %')

K-FOLD CROSS VALIDATION RESULTS FOR 10 FOLDS
--------------------------------
Fold 0: 0.784418202764977 %
Fold 1: 0.785426267281106 %
Fold 2: 0.7966589861751151 %
Fold 3: 0.7632848502304147 %
Fold 4: 0.7731494815668203 %
Fold 5: 0.7933107718894009 %
Fold 6: 0.7743735599078342 %
Fold 7: 0.7754896313364056 %
Fold 8: 0.7951636904761905 %
Fold 9: 0.7919642857142858 %
Average: 0.783323972734255 %


## Inference

In [17]:
test_dataset_dir = "/opt/ml/input/data/test/test.tsv"
test_data = load_data(test_dataset_dir)
test_label = test_data['label'].values

In [18]:
tokenized_test = tokenized_dataset(test_data, tokenizer)
test_dataset = RE_Dataset(tokenized_test ,test_label)

In [19]:
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=25,
    shuffle=False
)

In [20]:
oof_pred = None

for i in [1, 2, 5, 8, 9]:
    model.load_state_dict(torch.load(f'/opt/ml/model/model_state_dict{i}.pt'))
    model.eval()

    fold_pred = []
    for i, batch in enumerate(test_dataloader):
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
#                 token_type_ids=data['token_type_ids'].to(device)
            )
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()

        fold_pred.append(logits)

    if oof_pred is None:
        oof_pred = np.array(fold_pred) / 5
    else:
        oof_pred += (np.array(fold_pred) / 5)

In [21]:
oof_pred.shape

(40, 25, 42)

In [22]:
pred = np.argmax(oof_pred, axis=2)

In [23]:
submission = [pred[i][j] for i in range(len(pred)) for j in range(len(pred[i]))]
submission = pd.DataFrame(submission, columns=['pred'])
submission.to_csv('/opt/ml/prediction/submission.csv', index=False)

In [ ]:
from IPython.display import Audio
Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg", autoplay=True)